In [30]:
# Auteure : Safae SEBAY - Zara Zulfiqar 
# BUT3 SD EMS FA - G32
# Objectif du Notebook : maintenant que nous avons migré nos données sur mongodb en 2 collections différentes (Employés + Customers), 
# nous allons exécuter des requêtes mongodb pour visualisers les collections et documents, ainsi que vérifier la bonne structure des données 
import pymongo
import pandas as pd
# Connexion à MongoDB
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
db = mongo_client["ClassicModelDB"]

######### Dans un premier temps on repète les requêtes SQL faites à la séance 1 pour s'assurer que la migration a bien fonctionné ######
####### nous referons les questions 1 à 5 ainsi que la 8 ########
# 1. Lister les clients n'ayant jamais effectué de commande
client_sans_commande = db.Customers.aggregate([
    { "$match": { 
            "orders": { "$size": 0 }} },
    { "$project": {
            "_id":0,
            "customerNumber": 1,
            "customerName": 1}}])

client_sans_commande = pd.DataFrame(list(client_sans_commande))
print(len(client_sans_commande))
display(client_sans_commande)

## on voit qu'il y a bien 23 employés n'ayant jamais fait de commandes 

48


,customerNumber,customerName
0,125,Havel & Zbyszek Co
1,168,American Souvenirs Inc
2,169,Porto Imports Co.
3,206,"Asian Shopping Network, Co"
4,223,Natürlich Autos
5,237,ANG Resellers
6,247,Messner Shopping Network
7,273,"Franken Gifts, Co"
8,293,BG&E Collectables
9,303,Schuyler Imports


In [29]:
##Pour chaque produit, donner le nombre de commandes, la quantité totale commandée, et le nombre de clients différents ;

import pandas as pd

pipeline = [
    #les commandes et leurs détails
    {"$unwind": {"path": "$orders", "preserveNullAndEmptyArrays": True}},
    {"$unwind": {"path": "$orders.orderDetails", "preserveNullAndEmptyArrays": True}},

    #infos necesasaires
    {"$project": {
        "productCode": "$orders.orderDetails.productCode",
        "quantityOrdered": "$orders.orderDetails.quantityOrdered",
        "customerNumber": "$customerNumber"  # Associe chaque commande au client
    }},

    # Grouper lesw resultats par produit
    {"$group": {
        "_id": "$productCode",  # Groupe par code produit
        "nombre_commandes": {"$sum": 1},  # Compte le nombre de commandes
        "quantite_totale": {"$sum": "$quantityOrdered"},  # Somme des quantités commandées
        "clients_distincts": {"$addToSet": "$customerNumber"}  # Ensemble des clients distincts
    }},

    # Nombre de clients
    {"$project": {
        "productCode": "$_id",
        "nombre_commandes": 1,
        "quantite_totale": 1,
        "nombre_clients": {"$size": "$clients_distincts"}  # Taille de l'ensemble des clients
    }},

    # tri par quantite commandé
    {"$sort": {"quantite_totale": -1}}
]


result = list(mongo_db["Customers"].aggregate(pipeline))


df = pd.DataFrame(result)

display(df)


,_id,nombre_commandes,quantite_totale,productCode,nombre_clients
0,S18_3232,106,3616,S18_3232,40
1,S18_1342,56,2222,S18_1342,25
2,S18_3856,56,2152,S18_3856,26
3,S50_1341,56,2148,S50_1341,23
4,S700_4002,56,2146,S700_4002,26
...,...,...,...,...,...
105,S18_2248,50,1664,S18_2248,23
106,S24_3969,50,1648,S24_3969,21
107,S24_1046,50,1606,S24_1046,20
108,S18_4933,48,1534,S18_4933,20


In [35]:
###table de contigence du nombre de commande entre la ligne de produits et le pays du client


pipeline = [
    {"$unwind": {"path": "$orders", "preserveNullAndEmptyArrays": True}},
    {"$unwind": {"path": "$orders.orderDetails", "preserveNullAndEmptyArrays": True}},

    {"$lookup": {
        "from": "Products",  
        "localField": "orders.orderDetails.productCode",
        "foreignField": "productCode",
        "as": "productDetails"
    }},
    {"$unwind": {"path": "$productDetails", "preserveNullAndEmptyArrays": True}},

    {"$project": {
        "productLine": "$productDetails.productLine",  # Ligne de produit
        "country": "$address.country"  # Pays du client
    }},

    #Grouper par ligne de produits et pays
    {"$group": {
        "_id": {"productLine": "$productLine", "country": "$country"},
        "nombre_commandes": {"$sum": 1}
    }},

    {"$project": {
        "productLine": "$_id.productLine",
        "country": "$_id.country",
        "nombre_commandes": 1
    }}
]


result = list(mongo_db["Customers"].aggregate(pipeline))

display(result)


[{'$unwind': {'path': '$orders', 'preserveNullAndEmptyArrays': True}},
 {'$unwind': {'path': '$orders.orderDetails',
   'preserveNullAndEmptyArrays': True}},
 {'$lookup': {'from': 'Products',
   'localField': 'orders.orderDetails.productCode',
   'foreignField': 'productCode',
   'as': 'productDetails'}},
 {'$unwind': {'path': '$productDetails', 'preserveNullAndEmptyArrays': True}},
 {'$project': {'productLine': '$productDetails.productLine',
   'country': '$address.country'}},
 {'$group': {'_id': {'productLine': '$productLine', 'country': '$country'},
   'nombre_commandes': {'$sum': 1}}},
 {'$project': {'productLine': '$_id.productLine',
   'country': '$_id.country',
   'nombre_commandes': 1}}]

[{'_id': {'country': 'Sweden'}, 'nombre_commandes': 114, 'country': 'Sweden'},
 {'_id': {'country': 'Germany'},
  'nombre_commandes': 144,
  'country': 'Germany'},
 {'_id': {'country': 'Japan'}, 'nombre_commandes': 104, 'country': 'Japan'},
 {'_id': {'country': 'Ireland'}, 'nombre_commandes': 34, 'country': 'Ireland'},
 {'_id': {'country': 'Poland'}, 'nombre_commandes': 2, 'country': 'Poland'},
 {'_id': {'country': 'New Zealand'},
  'nombre_commandes': 298,
  'country': 'New Zealand'},
 {'_id': {'country': 'South Africa'},
  'nombre_commandes': 2,
  'country': 'South Africa'},
 {'_id': {'country': 'Israel'}, 'nombre_commandes': 2, 'country': 'Israel'},
 {'_id': {'country': 'UK'}, 'nombre_commandes': 288, 'country': 'UK'},
 {'_id': {'country': 'Austria'},
  'nombre_commandes': 110,
  'country': 'Austria'},
 {'_id': {'country': 'Italy'}, 'nombre_commandes': 242, 'country': 'Italy'},
 {'_id': {'country': 'Netherlands'},
  'nombre_commandes': 2,
  'country': 'Netherlands'},
 {'_id': {'coun